# Leetcode - similar problem and topic tag prediction

## Data extraction and Basic Preprocessing

In [1]:
import requests
import pandas as pd
import json
from tqdm import tqdm
import re
base_url = "https://leetcode.com/graphql/"

In [2]:
BASE_URL = "https://leetcode.com/graphql/"
QUERY = {"query":"\n    query problemsetQuestionList($categorySlug: String, $limit: Int, $skip: Int, $filters: QuestionListFilterInput) {\n  problemsetQuestionList: questionList(\n    categorySlug: $categorySlug\n    limit: $limit\n    skip: $skip\n    filters: $filters\n  ) {\n    total: totalNum\n    questions: data {\n      acRate\n      difficulty\n      freqBar\n      frontendQuestionId: questionFrontendId\n      isFavor\n      paidOnly: isPaidOnly\n      status\n      title\n      titleSlug\n      topicTags {\n        name\n        id\n        slug\n      }\n      hasSolution\n      hasVideoSolution\n    }\n  }\n}\n    ",
         "variables":{"categorySlug":"","skip":0,"limit":2214,"filters":{}}}
problems_data = requests.post(BASE_URL, json=QUERY).json()
print("Number of problems:", len(problems_data.get('data').get('problemsetQuestionList').get('questions')))

Number of problems: 2214


In [3]:
problems_data_list = problems_data.get('data').get('problemsetQuestionList').get('questions')

In [4]:
question_query = {"operationName":"questionData","variables":{"titleSlug":"longest-substring-of-one-repeating-character"},
         "query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    exampleTestcases\n    categoryTitle\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      hasVideoSolution\n      paidOnlyVideo\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    challengeQuestion {\n      id\n      date\n      incompleteChallengeCount\n      streakCount\n      type\n      __typename\n    }\n    __typename\n  }\n}\n"}
question_response = requests.post(BASE_URL, json=question_query).json()

In [7]:
def get_topics_formatted(topics_list):
    topics_retvar = []
    for topic in topics_list:
        topics_retvar.append(topic.get('name'))
    return topics_retvar

def get_slugs_and_topics(data_list):
    slugs = []
    topics = []
    for data in data_list:
        slugs.append(data.get('titleSlug'))
        topics.append(get_topics_formatted(data.get('topicTags')))
    return slugs, topics

In [8]:
slugs, topics = get_slugs_and_topics(problems_data_list)

In [9]:
def get_question_data(question_title_slug):
    row = []
    question_query = {"operationName":"questionData","variables":{"titleSlug":question_title_slug},
         "query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    exampleTestcases\n    categoryTitle\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      hasVideoSolution\n      paidOnlyVideo\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    challengeQuestion {\n      id\n      date\n      incompleteChallengeCount\n      streakCount\n      type\n      __typename\n    }\n    __typename\n  }\n}\n"}
    question_response = requests.post(BASE_URL, json=question_query).json().get('data').get('question')
    
    stats = json.loads(question_response.get('stats'))
    
    row.append(question_response.get('questionId'))
    row.append(question_response.get('title'))
    row.append(question_response.get('titleSlug'))
    row.append(question_response.get('content'))
    row.append(question_response.get('difficulty'))
    row.append(stats.get('acRate'))
    row.append(question_response.get('companyTagStats'))
    row.append(question_response.get('likes'))
    row.append(question_response.get('dislikes'))
    row.append(question_response.get('hints'))
    row.append(json.loads(question_response.get('similarQuestions')))
    row.append(stats.get('totalAcceptedRaw'))
    row.append(stats.get('totalSubmissionRaw'))
    return row

In [10]:
get_question_data('add-two-numbers')

['2',
 'Add Two Numbers',
 'add-two-numbers',
 '<p>You are given two <strong>non-empty</strong> linked lists representing two non-negative integers. The digits are stored in <strong>reverse order</strong>, and each of their nodes contains a single digit. Add the two numbers and return the sum&nbsp;as a linked list.</p>\n\n<p>You may assume the two numbers do not contain any leading zero, except the number 0 itself.</p>\n\n<p>&nbsp;</p>\n<p><strong>Example 1:</strong></p>\n<img alt="" src="https://assets.leetcode.com/uploads/2020/10/02/addtwonumber1.jpg" style="width: 483px; height: 342px;" />\n<pre>\n<strong>Input:</strong> l1 = [2,4,3], l2 = [5,6,4]\n<strong>Output:</strong> [7,0,8]\n<strong>Explanation:</strong> 342 + 465 = 807.\n</pre>\n\n<p><strong>Example 2:</strong></p>\n\n<pre>\n<strong>Input:</strong> l1 = [0], l2 = [0]\n<strong>Output:</strong> [0]\n</pre>\n\n<p><strong>Example 3:</strong></p>\n\n<pre>\n<strong>Input:</strong> l1 = [9,9,9,9,9,9,9], l2 = [9,9,9,9]\n<strong>Outp

### Note - We wont be able to extract the companies information as it is present for the leetcode and is not publically obtainable/scrapable

In [13]:
def create_dataset_raw(slugs, topics):
    data_raw = []
    for i in tqdm(range(len(slugs))):
        temp_data = get_question_data(slugs[i])
        temp_data.append(topics[i])
        data_raw.append(temp_data)
    df = pd.DataFrame(data_raw)
    return df

In [14]:
df = create_dataset_raw(slugs, topics)

100%|██████████████████████████████████████████████████████████████████████████████| 2214/2214 [21:38<00:00,  1.71it/s]


In [15]:
columns = [
    'question_id', 
    'question_title',
    'question_slug', 
    'question_text', 
    'difficulty',
    'success_rate', 
    'company_tags',
    'likes', 
    'dislikes',
    'hints', 
    'similar_questions',
    'total_accepted', 
    'total_submissions', 
    'topic_tagged_text'
    ]

In [16]:
df.columns = columns

In [17]:
df

,question_id,question_title,question_slug,question_text,difficulty,success_rate,company_tags,likes,dislikes,hints,similar_questions,total_accepted,total_submissions,topic_tagged_text
0,1,Two Sum,two-sum,<p>Given an array of integers <code>nums</code...,Easy,48.4%,None,30564,960,[A really brute force way would be to search f...,"[{'title': '3Sum', 'titleSlug': '3sum', 'diffi...",6255310,12924189,"[Array, Hash Table]"
1,2,Add Two Numbers,add-two-numbers,<p>You are given two <strong>non-empty</strong...,Medium,38.4%,None,17483,3629,[],"[{'title': 'Multiply Strings', 'titleSlug': 'm...",2644334,6888606,"[Linked List, Math, Recursion]"
2,3,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"<p>Given a string <code>s</code>, find the len...",Medium,32.9%,None,22404,1004,[],[{'title': 'Longest Substring with At Most Two...,3109972,9438634,"[Hash Table, String, Sliding Window]"
3,4,Median of Two Sorted Arrays,median-of-two-sorted-arrays,<p>Given two sorted arrays <code>nums1</code> ...,Hard,33.9%,None,15640,1936,[],[],1312772,3874658,"[Array, Binary Search, Divide and Conquer]"
4,5,Longest Palindromic Substring,longest-palindromic-substring,"<p>Given a string <code>s</code>, return <em>t...",Medium,31.7%,None,16733,981,[How can we reuse a previously computed palind...,"[{'title': 'Shortest Palindrome', 'titleSlug':...",1752820,5525765,"[String, Dynamic Programming]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2316,Count Hills and Valleys in an Array,count-hills-and-valleys-in-an-array,<p>You are given a <strong>0-indexed</strong> ...,Easy,55.9%,None,139,39,"[For each index, could you find the closest no...","[{'title': 'Find Peak Element', 'titleSlug': '...",15062,26955,[Array]
2210,2317,Count Collisions on a Road,count-collisions-on-a-road,<p>There are <code>n</code> cars on an infinit...,Medium,39.0%,None,222,140,[In what circumstances does a moving car not c...,"[{'title': 'Asteroid Collision', 'titleSlug': ...",11113,28479,"[String, Stack]"
2211,2318,Maximum Points in an Archery Competition,maximum-points-in-an-archery-competition,<p>Alice and Bob are opponents in an archery c...,Medium,46.6%,None,212,17,"[To obtain points for some certain section x, ...",[{'title': 'Maximum Product of the Length of T...,7261,15588,"[Array, Bit Manipulation, Recursion, Enumeration]"
2212,2319,Longest Substring of One Repeating Character,longest-substring-of-one-repeating-character,<p>You are given a <strong>0-indexed</strong> ...,Hard,27.2%,None,64,56,[Use a segment tree to perform fast point upda...,"[{'title': 'Merge Intervals', 'titleSlug': 'me...",1523,5602,"[Array, String, Segment Tree, Ordered Set]"


In [ ]:
df.to_csv('data_raw.csv');

In [20]:
def remove_html_tags(text):
    patterns_rep_with_space = [r'&nbsp', r'\t', r'&lt', r'.', r',']
    patterns_rep_without_space = [r'<.*?>', r'\n', r';', r'&quot', r']']
    for pattern in patterns_rep_with_space:
        p = re.compile(pattern)
        text = p.sub(" ", text)
    for pattern in patterns_rep_without_space:
        p = re.compile(pattern)
        text = p.sub("", text)
    text = re.sub(r'\s{2,}', '', text)
    return text

In [22]:
# preprocessing the question text
df['question_text'] = list(map(lambda x: remove_html_tags(x).split('Example')[0] if x!= None else None, df['question_text']))

In [23]:
df['success_rate'] = list(map(lambda x: float(str(x)[:-1]), df['success_rate']))

In [31]:
df.iloc[3].question_text

'Given two sorted arrays nums1 and nums2 of size m and n respectively, return the median of the two sorted arrays.The overall run time complexity should be O(log (m+n)). '

In [32]:
df.to_csv('data_raw_question_cleaned.csv');

In [33]:
df

,question_id,question_title,question_slug,question_text,difficulty,success_rate,company_tags,likes,dislikes,hints,similar_questions,total_accepted,total_submissions,topic_tagged_text
0,1,Two Sum,two-sum,Given an array of integers nums and an integer...,Easy,48.4,None,30564,960,[A really brute force way would be to search f...,"[{'title': '3Sum', 'titleSlug': '3sum', 'diffi...",6255310,12924189,"[Array, Hash Table]"
1,2,Add Two Numbers,add-two-numbers,You are given two non-empty linked lists repre...,Medium,38.4,None,17483,3629,[],"[{'title': 'Multiply Strings', 'titleSlug': 'm...",2644334,6888606,"[Linked List, Math, Recursion]"
2,3,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"Given a string s, find the length of the longe...",Medium,32.9,None,22404,1004,[],[{'title': 'Longest Substring with At Most Two...,3109972,9438634,"[Hash Table, String, Sliding Window]"
3,4,Median of Two Sorted Arrays,median-of-two-sorted-arrays,Given two sorted arrays nums1 and nums2 of siz...,Hard,33.9,None,15640,1936,[],[],1312772,3874658,"[Array, Binary Search, Divide and Conquer]"
4,5,Longest Palindromic Substring,longest-palindromic-substring,"Given a string s, return the longest palindrom...",Medium,31.7,None,16733,981,[How can we reuse a previously computed palind...,"[{'title': 'Shortest Palindrome', 'titleSlug':...",1752820,5525765,"[String, Dynamic Programming]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2316,Count Hills and Valleys in an Array,count-hills-and-valleys-in-an-array,You are given a 0-indexed integer array nums. ...,Easy,55.9,None,139,39,"[For each index, could you find the closest no...","[{'title': 'Find Peak Element', 'titleSlug': '...",15062,26955,[Array]
2210,2317,Count Collisions on a Road,count-collisions-on-a-road,There are n cars on an infinitely long road. T...,Medium,39.0,None,222,140,[In what circumstances does a moving car not c...,"[{'title': 'Asteroid Collision', 'titleSlug': ...",11113,28479,"[String, Stack]"
2211,2318,Maximum Points in an Archery Competition,maximum-points-in-an-archery-competition,Alice and Bob are opponents in an archery comp...,Medium,46.6,None,212,17,"[To obtain points for some certain section x, ...",[{'title': 'Maximum Product of the Length of T...,7261,15588,"[Array, Bit Manipulation, Recursion, Enumeration]"
2212,2319,Longest Substring of One Repeating Character,longest-substring-of-one-repeating-character,You are given a 0-indexed string s. You are al...,Hard,27.2,None,64,56,[Use a segment tree to perform fast point upda...,"[{'title': 'Merge Intervals', 'titleSlug': 'me...",1523,5602,"[Array, String, Segment Tree, Ordered Set]"


In [60]:
## obtaining unique topics - 

unique_topics = df['topic_tagged_text'].values

import numpy as np
unique_topics = np.array(unique_topics)

def get_topics_set(topics):
    topics_set = set({})
    topics_values = []
    for i in topics:
        temp = []
        for j in i:
            temp.append(j)
            if j not in topics_set:
                topics_set.add(j)
        topics_values.append(temp)
    return topics_set, topics_values

unique_topics_final, topics_rows = get_topics_set(unique_topics)

print("Total Unique Topics found : {}".format(len(unique_topics_final)))

Total Unique Topics found : 71


In [65]:
## Obtaining slugs for similar questions

def get_similar_question_slugs(similar_questions):
    similar_question_slugs = []
    for one_set in similar_questions:
        slugs = []
        for one_dict in one_set:
            slugs.append(one_dict.get('titleSlug'))
        similar_question_slugs.append(slugs)
    return similar_question_slugs

similar_ques_slugs_final = get_similar_question_slugs(df['similar_questions'])

In [75]:
## Obtaining hints texts and cleaning

hints_raw = df['hints'].values

def preprocess_hints(hints):
    hints_retvar = []
    for hint_row in hints:
        temp_row = []
        for hint in hint_row:
            temp_row.append(remove_html_tags(hint))
        hints_retvar.append(temp_row)
    return hints_retvar

hints_processed = preprocess_hints(hints_raw)

In [176]:
## Getting data ready for SIMILAR QUESTION PREDICTION

data_set1 = df.copy()

data_set1['similar_question_slug'] = similar_ques_slugs_final

# Creating similar questions data
def create_similar_questions_data(df):
    indices = np.arange(0,len(df),1)
    similar_questions = []
    for row in range(len(df)):
        
        similar_indices = []
        qid1 = df.iloc[row]['question_id']
        q1 = df.iloc[row]['question_text']
        q1_topics = df.iloc[row]['topic_tagged_text']
        if q1 is None:
            continue
        
        ## Generate dataset for positive label - Question 1 and Question 2 are similar
        for i in df.iloc[row]['similar_question_slug']:
            idx = np.where(df['question_slug'] == i)[0]
            if len(idx) == 0:
                continue
            idx = idx[0]
            q2 = df.iloc[idx]['question_text']
            if q2 is None:
                continue
            q2_topics = df.iloc[idx]['topic_tagged_text']
            qid2 = df.iloc[idx]['question_id']
            similar_questions.append([qid1, qid2, q1, q2, q1_topics, q2_topics, 1])
            similar_indices.append(idx)
            
        # Generating dataset for negative label - Question 1 and Question 2 are not similar
        set_include = set(indices)
        set_remove = set(similar_indices)
        set_include = set_include.difference(set_remove)
        include_idx = list(set_include)
        neg_points_idx = np.random.choice(include_idx, len(similar_indices), replace = False)
        for i in neg_points_idx:
            qid2 = df.iloc[i]['question_id']
            q2 = df.iloc[i]['question_text']
            q2_topics = df.iloc[i]['topic_tagged_text']
            if q2 is None:
                continue
            similar_questions.append([qid1, qid2, q1, q2, q1_topics, q2_topics,  0])
        
    return similar_questions

similar_questions_data = create_similar_questions_data(data_set1)
similar_questions_data_columns = ['q1id', 'q2id', 'question1', 'question2', 'q1_topics', 'q2_topics', 'target']
similar_questions_df = pd.DataFrame(similar_questions_data, columns = similar_questions_data_columns)


## Shuffling 
from sklearn.utils import shuffle
similar_questions_df = shuffle(similar_questions_df)
similar_questions_df.reset_index(inplace = True)
similar_questions_df.drop('index', axis=1, inplace=True)


## Saving the data
## Converting data slightly to make it easier to save and retrieve
similar_questions_df_copy = similar_questions_df.copy()
similar_questions_df_copy['q1_topics'] = list(map(lambda x: ",".join(x), similar_questions_df_copy['q1_topics']))
similar_questions_df_copy['q2_topics'] = list(map(lambda x: ",".join(x), similar_questions_df_copy['q2_topics']))
similar_questions_df_copy.to_csv('similar_question_data_with_topics.csv')
similar_questions_df_copy.head()

,q1id,q2id,question1,question2,q1_topics,q2_topics,target
0,88,1019,You are given two integer arrays nums1 and num...,Given an integer array nums sorted in non-decr...,"Array,Two Pointers,Sorting","Array,Two Pointers,Sorting",1
1,724,2102,"Given an array of integers nums, calculate the...","Given a 0-indexed integer array nums, find the...","Array,Prefix Sum","Array,Prefix Sum",1
2,967,1022,"Given an n x n array of integers matrix, retur...",You are given an m x n integer array grid wher...,"Array,Dynamic Programming,Matrix","Array,Backtracking,Bit Manipulation,Matrix",0
3,300,673,"Given an integer array nums, return the length...","Given an integer array nums, return the number...","Array,Binary Search,Dynamic Programming","Array,Dynamic Programming,Binary Indexed Tree,...",1
4,2280,334,You are given two 0-indexed arrays nums1 and n...,"Given an integer array nums, return true if th...","Array,Binary Search,Divide and Conquer,Binary ...","Array,Greedy",1


### As of now we have the raw data that we can use for creating a classifier for predicting if two questions are similar or not
### Note - I have done basic preprocessing for now, more will be done in individual notebooks for the two problems that we have <br/>1. predicting tags for the questions<br/>2. Classifying if questions are similar or not

### Now lets get the data ready for predicting topic tags